In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
df = pd.read_csv('heart.csv')

In [3]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [4]:
df.shape

(918, 12)

In [5]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [6]:
col = ['Age','RestingBP','Cholesterol','FastingBS','MaxHR','Oldpeak']
col

['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']

### Calculating z-score

In [7]:
col_mean = df[col].mean()
col_std = df[col].std()

z_score = (df[col] - col_mean)/col_std

z_threshold = 3

In [8]:
newdf = df
newdf.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [9]:
for i in z_score:
    newdf['Z_score'+str(i)]=z_score[i]

In [10]:
newdf.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Z_scoreAge,Z_scoreRestingBP,Z_scoreCholesterol,Z_scoreFastingBS,Z_scoreMaxHR,Z_scoreOldpeak
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0,-1.432359,0.410685,0.824621,-0.551041,1.382175,-0.831979
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1,-0.478223,1.490940,-0.171867,-0.551041,0.753746,0.105606
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0,-1.750404,-0.129442,0.769768,-0.551041,-1.524307,-0.831979
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1,-0.584238,0.302660,0.138964,-0.551041,-1.131539,0.574398
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0,0.051853,0.950812,-0.034736,-0.551041,-0.581664,-0.831979


### Removing the outlier with the threshold of z-score greater than 3

In [11]:
outliers = np.any(np.abs(z_score) > z_threshold,axis=1)
df1 = df[~outliers]

In [12]:
df1

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Z_scoreAge,Z_scoreRestingBP,Z_scoreCholesterol,Z_scoreFastingBS,Z_scoreMaxHR,Z_scoreOldpeak
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0,-1.432359,0.410685,0.824621,-0.551041,1.382175,-0.831979
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1,-0.478223,1.490940,-0.171867,-0.551041,0.753746,0.105606
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0,-1.750404,-0.129442,0.769768,-0.551041,-1.524307,-0.831979
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1,-0.584238,0.302660,0.138964,-0.551041,-1.131539,0.574398
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0,0.051853,0.950812,-0.034736,-0.551041,-0.581664,-0.831979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1,-0.902283,-1.209697,0.596068,-0.551041,-0.188897,0.293123
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1,1.536064,0.626736,-0.053020,1.812770,0.164595,2.355810
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1,0.369898,-0.129442,-0.619830,-0.551041,-0.856602,0.293123
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1,0.369898,-0.129442,0.340090,-0.551041,1.460728,-0.831979


In [13]:
for i in range(2):
    print(df.shape[i]-df1.shape[i])

19
0


### We have successfully removed 19 outliers from the dataset

In [14]:
df1.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease', 'Z_scoreAge', 'Z_scoreRestingBP', 'Z_scoreCholesterol',
       'Z_scoreFastingBS', 'Z_scoreMaxHR', 'Z_scoreOldpeak'],
      dtype='object')

In [15]:
columns_to_remove = ['Z_scoreAge', 'Z_scoreRestingBP', 'Z_scoreCholesterol',
                     'Z_scoreFastingBS', 'Z_scoreMaxHR', 'Z_scoreOldpeak']

# Drop the specified columns
df1 = df1.drop(columns=columns_to_remove)

In [16]:
df1.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [17]:
df1.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')

In [18]:
categorical_columns = ['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']

In [19]:
df2 = pd.get_dummies(df1, columns=categorical_columns, drop_first=True)
df2.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,1,1,0,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,1,1,0,0,0,1,0,0,1
3,48,138,214,0,108,1.5,1,0,0,0,0,1,0,1,1,0
4,54,150,195,0,122,0.0,0,1,0,1,0,1,0,0,0,1


In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [21]:
y = df2['HeartDisease']
df2.drop(columns=['HeartDisease'], inplace=True)

In [22]:
X = df2

In [23]:
X_scaled = scaler.fit_transform(X)


In [24]:
X_scaled.shape

(899, 15)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_predict = train_test_split(X_scaled,y,test_size=0.35,random_state=42)

In [26]:
X_valid.shape

(315, 15)

In [27]:
X_train.shape

(584, 15)

In [28]:
from sklearn.svm import SVC 
model=SVC(kernel='rbf')
model.fit(X_train,y_train)
model.score(X_valid,y_predict)

0.8539682539682539

In [38]:
from sklearn.model_selection import cross_val_score
sv = cross_val_score(SVC(kernel='rbf'),X_train,y_train)
sv

array([0.92307692, 0.87179487, 0.84615385, 0.88034188, 0.84482759])

In [39]:
np.average(sv)*100

87.32390215148837

In [40]:
lr = cross_val_score(LogisticRegression(),X_train,y_train)
print(lr)
np.average(lr)*100

[0.93162393 0.81196581 0.82905983 0.88888889 0.85344828]


86.29973474801062

In [41]:
rclf = cross_val_score(RandomForestClassifier(),X_train,y_train)
print(rclf)
np.average(rclf)*100

[0.8974359  0.90598291 0.84615385 0.8974359  0.85344828]


88.00913645741232

In [29]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'gamma': [0.1, 1, 10],
    'degree': [2, 3, 4]
}

# Initialize the SVM model
svm_model = SVC(kernel='poly')

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')

# Perform grid search on the training data
grid_search.fit(X_train, y_train.values.ravel())

# Get the best parameters and corresponding accuracy score
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Accuracy:", best_accuracy)

Best Parameters: {'degree': 3, 'gamma': 0.1}
Best Accuracy: 0.8235879813199402


In [30]:
from sklearn.linear_model import LogisticRegression
log_model=LogisticRegression()
log_model.fit(X_train,y_train)
log_model.score(X_valid,y_predict)

0.8444444444444444

In [31]:
from sklearn.ensemble import RandomForestClassifier
rf_model=RandomForestClassifier(max_depth=12, random_state=0)
rf_model.fit(X_train,y_train)
rf_model.score(X_valid,y_predict)

0.8603174603174604

In [32]:
X

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,1,1,0,0,1,0,0,0,1
1,49,160,180,0,156,1.0,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,1,1,0,0,0,1,0,0,1
3,48,138,214,0,108,1.5,0,0,0,0,1,0,1,1,0
4,54,150,195,0,122,0.0,1,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,110,264,0,132,1.2,1,0,0,1,1,0,0,1,0
914,68,144,193,1,141,3.4,1,0,0,0,1,0,0,1,0
915,57,130,131,0,115,1.2,1,0,0,0,1,0,1,1,0
916,57,130,236,0,174,0.0,0,1,0,0,0,0,0,1,0


In [33]:
X.shape

(899, 15)

### Now let's do PCA for decomposition or high dimensionality reduction in features

In [34]:
from sklearn.decomposition import PCA
pca = PCA(0.99)
X_pca = pca.fit_transform(X)
X_pca.shape

(899, 3)

In [35]:
pca.explained_variance_ratio_

array([0.92112603, 0.0506488 , 0.0225813 ])

In [36]:
X_trainpca,X_validpca,ytrainpca,yvalidpca=train_test_split(X_pca,y,test_size=0.35,random_state=42)
mdl = SVC(kernel='rbf')
mdl.fit(X_trainpca,ytrainpca)
mdl.score(X_validpca,yvalidpca)

0.7015873015873015

In [37]:
mfl = RandomForestClassifier(max_depth=12, random_state=0)
rf_model.fit(X_trainpca,ytrainpca)
rf_model.score(X_validpca,yvalidpca)

0.6793650793650794